In [1]:
import cv2
import numpy as np
import time
import threading
import math
import random
from collections import Counter
from IPython.display import clear_output

import sys, os
sys.path.append("../x11-window-interactor")
from x11_interactor import X11WindowInteractor

In [2]:
# Initialize mouse and keyboard controllers
interactor = X11WindowInteractor()

# Global variable to control the start and stop of the script
script_running = False

# Main Roi
main_roi = {'left': 0, 'top': 0, 'width': 3840, 'height': 2160}

# HOH Presurge (ROI)
hoh_presurge_roi = {'left': 1600, 'top': 200, 'width': 200, 'height': 200}

# Dark Portal postsurge
dark_portal_postsurge = {'left':1650, 'top':815, 'width':60, 'height':60}

Click on the target window after running this...


In [6]:
def find_image(template_path, screenshot):
    # Load the template image and convert to grayscale
    template = cv2.imread(template_path, cv2.IMREAD_GRAYSCALE)
    template_edges = cv2.Canny(template, 0, 100)

    # Convert screenshot to grayscale
    gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY)
    screenshot_edges = cv2.Canny(gray, 0, 100)

    # Perform template matching using the edge images
    result = cv2.matchTemplate(screenshot_edges, template_edges, cv2.TM_CCOEFF_NORMED)
    _, max_val, _, max_loc = cv2.minMaxLoc(result)
    # print(max_val)
    # Get the coordinates and dimensions of the matched region
    h, w = template.shape
    top_left = max_loc
    return top_left[0], top_left[1], w, h

In [7]:
def randomize_click_position(center_x, center_y, width, height, shape='rectangle', roi_diminish=2):
    if shape == 'circle':
        # Randomize within a circle
        radius = (min(width, height) // 2) // roi_diminish
        angle = np.random.uniform(0, 2 * math.pi)
        r = radius * np.sqrt(np.random.uniform(0, 1))
        click_x = int(center_x + r * math.cos(angle))
        click_y = int(center_y + r * math.sin(angle))
    else:
        # Randomize within a rectangle
        area_width = width * 0.5
        area_height = height * 0.5

        left_bound = center_x - area_width / roi_diminish
        right_bound = center_x + area_width / roi_diminish
        top_bound = center_y - area_height / roi_diminish
        bottom_bound = center_y + area_height / roi_diminish

        click_x = int(np.random.normal(center_x, area_width / 6))
        click_y = int(np.random.normal(center_y, area_height / 6))

        click_x = min(max(click_x, left_bound), right_bound)
        click_y = min(max(click_y, top_bound), bottom_bound)

    return int(click_x), int(click_y)

In [3]:
cv2.namedWindow("test", cv2.WINDOW_NORMAL)
interactor.activate()
while True:
    img = interactor.capture()
    cv2.imshow("test", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Interrupted by user.")
        break
cv2.destroyAllWindows()

Interrupted by user.
